In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
url = "https://www.basketball-reference.com/leagues/NBA_2023.html#all_per_game_team-opponent"

In [3]:
html = urlopen(url)
soup = BeautifulSoup(html)

# DATAFRAME

In [5]:
for table in soup.find_all('table'):
    print(table.get('id'))

confs_standings_E
confs_standings_W
divs_standings_E
divs_standings_W
per_game-team
per_game-opponent
totals-team
totals-opponent
per_poss-team
per_poss-opponent
advanced-team
shooting-team
shooting-opponent


In [6]:
table = soup.find(lambda tag: tag.name =='table' and tag.get('id')=='advanced-team')

In [11]:
columns = table.findAll(lambda tag: tag.name=='tr',limit=2)
columns

[<tr class="over_header"> <th aria-label="" class="over_header center" colspan="18" data-stat=""></th> <th aria-label="" class="over_header center" colspan="4" data-stat="ff_off">Offense Four Factors</th><th></th> <th aria-label="" class="over_header center" colspan="4" data-stat="ff_def">Defense Four Factors</th> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th> </tr>,
 <tr> <th aria-label="Rk" class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th> <th aria-label="team" class="poptip center" data-stat="team" scope="col">Team</th> <th aria-label="Age" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th> <th aria-label="Wins" class="poptip center" data-stat="wins" data-tip="Wins" scope="col">W</th> <th aria-label="Losses" class="poptip center" data-stat="losses" data-tip="Losses" scope="col">L</th> <th aria-label="PW

In [44]:
headers = [th.getText() for th in columns[1].findAll('th')]
headers = headers[1:]
headers

['Team',
 'Age',
 'W',
 'L',
 'PW',
 'PL',
 'MOV',
 'SOS',
 'SRS',
 'ORtg',
 'DRtg',
 'NRtg',
 'Pace',
 'FTr',
 '3PAr',
 'TS%',
 '\xa0',
 'eFG%',
 'TOV%',
 'ORB%',
 'FT/FGA',
 '\xa0',
 'eFG%',
 'TOV%',
 'DRB%',
 'FT/FGA',
 '\xa0',
 'Arena',
 'Attend.',
 'Attend./G']

In [45]:
rows = table.tbody.findAll('tr')[0:]

In [46]:
team_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

In [47]:
df = pd.DataFrame(team_stats, columns = headers)
df.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,FT/FGA,,eFG%,TOV%,DRB%,FT/FGA,,Arena,Attend.,Attend./G
0,Boston Celtics,27.4,26,10,25,11,6.78,-0.33,6.45,118.3,...,.211,,.529,12.0,77.2,.184,,TD Garden,"363,964","18,198"
1,New Orleans Pelicans,25.7,23,12,23,12,5.40,-0.32,5.08,116.0,...,.229,,.538,14.0,77.1,.196,,Smoothie King Center,"329,295","16,465"
2,Cleveland Cavaliers,25.4,22,14,24,12,5.19,-0.19,5.01,114.6,...,.220,,.532,13.8,78.1,.200,,Rocket Mortgage Fieldhouse,"369,208","18,460"
3,Memphis Grizzlies,24.6,21,13,21,13,3.85,0.14,3.99,113.9,...,.195,,.521,12.6,76.1,.201,,FedEx Forum,"274,678","17,167"
4,Philadelphia 76ers,28.5,20,14,21,13,3.29,-0.37,2.92,113.5,...,.236,,.528,14.2,75.9,.219,,Wells Fargo Center,"382,772","20,146"


In [65]:
df.shape

(30, 27)

## Cleaning

In [49]:
df.isnull().sum()

Team         0
Age          0
W            0
L            0
PW           0
PL           0
MOV          0
SOS          0
SRS          0
ORtg         0
DRtg         0
NRtg         0
Pace         0
FTr          0
3PAr         0
TS%          0
             0
eFG%         0
TOV%         0
ORB%         0
FT/FGA       0
             0
eFG%         0
TOV%         0
DRB%         0
FT/FGA       0
             0
Arena        0
Attend.      0
Attend./G    0
dtype: int64

In [54]:
df.dtypes

Team         object
Age          object
W            object
L            object
PW           object
PL           object
MOV          object
SOS          object
SRS          object
ORtg         object
DRtg         object
NRtg         object
Pace         object
FTr          object
3PAr         object
TS%          object
             object
eFG%         object
TOV%         object
ORB%         object
FT/FGA       object
             object
eFG%         object
TOV%         object
DRB%         object
FT/FGA       object
             object
Arena        object
Attend.      object
Attend./G    object
dtype: object

In [64]:
df = df.drop('\xa0',axis=1)
df.columns

Index(['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%',
       'FT/FGA', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'Arena', 'Attend.',
       'Attend./G'],
      dtype='object')

In [71]:
df.columns = ['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'Off - eFG%', 'Off - TOV%', 'Off - ORB%',
       'Off - FT/FGA', 'Def - eFG%', 'Def - TOV%', 'Def - DRB%', 'Def - FT/FGA', 'Arena', 'Attend.',
       'Attend./G']

# Stats

### Margin Of Victory

In [89]:
df.sort_values(by=['MOV'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'MOV','W', 'L', 'PW', 'PL', 'SOS', 'SRS', 'ORtg','DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%']].head(5)

,Team,MOV,W,L,PW,PL,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%
1,Boston Celtics,6.78,26,10,25,11,-0.33,6.45,118.3,111.6,+6.7,99.3,.255,.479,.607
2,New Orleans Pelicans,5.40,23,12,23,12,-0.32,5.08,116.0,110.7,+5.3,100.0,.292,.347,.587
3,Cleveland Cavaliers,5.19,22,14,24,12,-0.19,5.01,114.6,109.3,+5.3,95.6,.278,.371,.588
4,Memphis Grizzlies,3.85,21,13,21,13,0.14,3.99,113.9,110.1,+3.8,100.6,.272,.362,.559
5,Philadelphia 76ers,3.29,20,14,21,13,-0.37,2.92,113.5,110.2,+3.3,97.3,.288,.397,.596


### Strength of Schedule
> The rating is denominated in points above/below average, where zero is average. A positive number indicates a harder than average schedule

In [92]:
df.sort_values(by=['SOS'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'SOS','W', 'L', 'PW', 'PL','MOV', 'SRS','ORtg','DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%']].head(5)

,Team,SOS,W,L,PW,PL,MOV,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%
1,Detroit Pistons,0.99,9,29,11,27,-7.26,-6.28,111.4,118.7,-7.3,99.3,.312,.382,.560
2,Chicago Bulls,0.61,16,19,16,19,-1.03,-0.42,112.8,113.8,-1.0,100.0,.256,.329,.585
3,Toronto Raptors,0.54,16,20,18,18,0.08,0.63,114.0,113.9,+0.1,97.1,.278,.355,.553
4,Washington Wizards,0.43,16,21,17,20,-0.97,-0.54,112.6,113.6,-1.0,98.8,.266,.367,.579
5,Dallas Mavericks,0.36,20,16,21,15,2.28,2.64,115.3,113.0,+2.3,95.9,.307,.500,.592


### Simple Rating System
> A rating that takes into account average point differential and strength of schedule. The rating is denominated in points above/below average, where zero is average.

In [94]:
df.sort_values(by=['MOV'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'SRS','W', 'L', 'PW', 'PL','MOV', 'SOS',  'ORtg','DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%']].head(5)

,Team,SRS,W,L,PW,PL,MOV,SOS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%
1,Boston Celtics,6.45,26,10,25,11,6.78,-0.33,118.3,111.6,+6.7,99.3,.255,.479,.607
2,New Orleans Pelicans,5.08,23,12,23,12,5.40,-0.32,116.0,110.7,+5.3,100.0,.292,.347,.587
3,Cleveland Cavaliers,5.01,22,14,24,12,5.19,-0.19,114.6,109.3,+5.3,95.6,.278,.371,.588
4,Memphis Grizzlies,3.99,21,13,21,13,3.85,0.14,113.9,110.1,+3.8,100.6,.272,.362,.559
5,Philadelphia 76ers,2.92,20,14,21,13,3.29,-0.37,113.5,110.2,+3.3,97.3,.288,.397,.596


## Offensive

### Offensive Rating
> For teams it is points scored per 100 possessions

In [96]:
df.sort_values(by=['ORtg'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'ORtg','Off - eFG%', 'Off - TOV%', 'Off - ORB%','Off - FT/FGA', '3PAr', 'TS%']].head(5)

,Team,ORtg,Off - eFG%,Off - TOV%,Off - ORB%,Off - FT/FGA,3PAr,TS%
1,Boston Celtics,118.3,.569,12.2,21.2,.211,.479,.607
2,Denver Nuggets,117.2,.577,13.7,24.6,.199,.349,.605
3,Phoenix Suns,116.9,.544,12.2,27.7,.183,.380,.577
4,Sacramento Kings,116.1,.566,13.0,20.8,.224,.425,.601
5,Utah Jazz,116.1,.553,13.3,26.5,.197,.444,.587


- Effective Field Goal Percentage; the formula is (FG + 0.5 * 3P) / FGA. This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal. For example, suppose Player A goes 4 for 10 with 2 threes, while Player B goes 5 for 10 with 0 threes. Each player would have 10 points from field goals, and thus would have the same effective field goal percentage (50%)

In [83]:
df.sort_values(by=['Off - eFG%'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'ORtg','Off - eFG%', 'Off - TOV%', 'Off - ORB%','Off - FT/FGA', '3PAr', 'TS%']].head(5)

,Team,Off - eFG%,Off - TOV%,Off - ORB%,Off - FT/FGA
1,Brooklyn Nets,.582,13.6,19.5,.199
2,Denver Nuggets,.577,13.7,24.6,.199
3,Boston Celtics,.569,12.2,21.2,.211
4,Sacramento Kings,.566,13.0,20.8,.224
5,Golden State Warriors,.564,14.6,22.4,.178


- TOV% - Turnover Percentage (available since the 1977-78 season in the NBA); the formula is 100 * TOV / (FGA + 0.44 * FTA + TOV). Turnover percentage is an estimate of turnovers per 100 plays.

In [106]:
df.sort_values(by=['Off - TOV%'],ascending=True,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Off - TOV%', 'ORtg','Off - eFG%', 'Off - ORB%','Off - FT/FGA', '3PAr', 'TS%']].head(5)

,Team,Off - TOV%,ORtg,Off - eFG%,Off - ORB%,Off - FT/FGA,3PAr,TS%
1,Toronto Raptors,11.0,114.0,.510,27.4,.220,.355,.553
2,Atlanta Hawks,11.2,112.5,.522,23.6,.193,.335,.560
3,Dallas Mavericks,11.8,115.3,.559,18.8,.227,.500,.592
4,Charlotte Hornets,12.0,108.9,.504,26.5,.187,.354,.538
5,New York Knicks,12.0,115.0,.523,28.4,.218,.383,.563


- ORB% - Offensive Rebound Percentage (available since the 1970-71 season in the NBA); the formula is 100 * (ORB * (Tm MP / 5)) / (MP * (Tm ORB + Opp DRB)). Offensive rebound percentage is an estimate of the percentage of available offensive rebounds a player grabbed while he was on the floor.

In [101]:
df.sort_values(by=['Off - ORB%'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Off - ORB%', 'ORtg','Off - eFG%', 'Off - TOV%','Off - FT/FGA', '3PAr', 'TS%']].head(5)

,Team,Off - ORB%,ORtg,Off - eFG%,Off - TOV%,Off - FT/FGA,3PAr,TS%
1,Houston Rockets,30.2,110.2,.510,14.6,.223,.393,.552
2,Memphis Grizzlies,29.0,113.9,.528,12.3,.195,.362,.559
3,New York Knicks,28.4,115.0,.523,12.0,.218,.383,.563
4,Phoenix Suns,27.7,116.9,.544,12.2,.183,.380,.577
5,Toronto Raptors,27.4,114.0,.510,11.0,.220,.355,.553


- FT - Free Throws
- FGA - Field Goal Attempts 

In [102]:
df.sort_values(by=['Off - FT/FGA'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team','Off - FT/FGA', 'ORtg','Off - eFG%', 'Off - TOV%', 'Off - ORB%', '3PAr', 'TS%']].head(5)

,Team,Off - FT/FGA,ORtg,Off - eFG%,Off - TOV%,Off - ORB%,3PAr,TS%
1,Detroit Pistons,.242,111.4,.516,12.8,24.9,.382,.560
2,Philadelphia 76ers,.236,113.5,.553,13.4,19.6,.397,.596
3,Orlando Magic,.232,110.5,.529,14.1,24.0,.362,.572
4,New Orleans Pelicans,.229,116.0,.548,13.2,26.7,.347,.587
5,Dallas Mavericks,.227,115.3,.559,11.8,18.8,.500,.592


## Defense

In [115]:
df.sort_values(by=['DRtg'],ascending=True,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'DRtg','Def - eFG%', 'Def - TOV%', 'Def - DRB%', 'Def - FT/FGA']].head(5)

,Team,DRtg,Def - eFG%,Def - TOV%,Def - DRB%,Def - FT/FGA
1,Cleveland Cavaliers,109.3,.532,13.8,78.1,.200
2,Milwaukee Bucks,110.1,.519,11.2,78.3,.183
3,Memphis Grizzlies,110.1,.521,12.6,76.1,.201
4,Philadelphia 76ers,110.2,.528,14.2,75.9,.219
5,Los Angeles Clippers,110.3,.520,11.9,76.7,.186


In [117]:
df.sort_values(by=['Def - eFG%'],ascending=True,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Def - eFG%','DRtg', 'Def - TOV%', 'Def - DRB%', 'Def - FT/FGA']].head(5)

,Team,Def - eFG%,DRtg,Def - TOV%,Def - DRB%,Def - FT/FGA
1,Milwaukee Bucks,.519,110.1,11.2,78.3,.183
2,Los Angeles Clippers,.520,110.3,11.9,76.7,.186
3,Memphis Grizzlies,.521,110.1,12.6,76.1,.201
4,Brooklyn Nets,.525,112.7,12.6,74.5,.217
5,Philadelphia 76ers,.528,110.2,14.2,75.9,.219


In [122]:
df.sort_values(by=['Def - TOV%'],ascending=True,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Def - TOV%','DRtg','Def - eFG%',  'Def - DRB%', 'Def - FT/FGA']].head(5)

,Team,Def - TOV%,DRtg,Def - eFG%,Def - DRB%,Def - FT/FGA
1,Los Angeles Lakers,11.0,114.5,.540,76.0,.171
2,Milwaukee Bucks,11.2,110.1,.519,78.3,.183
3,Washington Wizards,11.4,113.6,.530,77.4,.209
4,Los Angeles Clippers,11.9,110.3,.520,76.7,.186
5,New York Knicks,11.9,113.1,.528,75.4,.209


In [124]:
df.sort_values(by=['Def - DRB%'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Def - DRB%','DRtg','Def - eFG%', 'Def - TOV%', 'Def - FT/FGA']].head(5)

,Team,Def - DRB%,DRtg,Def - eFG%,Def - TOV%,Def - FT/FGA
1,Milwaukee Bucks,78.3,110.1,.519,11.2,.183
2,Sacramento Kings,78.3,114.5,.552,12.8,.220
3,Cleveland Cavaliers,78.1,109.3,.532,13.8,.200
4,Denver Nuggets,78.1,115.2,.558,12.5,.205
5,Chicago Bulls,78.0,113.8,.555,13.7,.203


In [127]:
df.sort_values(by=['Def - FT/FGA'],ascending=True,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Def - FT/FGA', 'DRtg','Def - eFG%', 'Def - TOV%', 'Def - DRB%']].head(5)

,Team,Def - FT/FGA,DRtg,Def - eFG%,Def - TOV%,Def - DRB%
1,Los Angeles Lakers,.171,114.5,.540,11.0,76.0
2,Miami Heat,.182,111.5,.557,15.2,77.6
3,Milwaukee Bucks,.183,110.1,.519,11.2,78.3
4,Boston Celtics,.184,111.6,.529,12.0,77.2
5,Los Angeles Clippers,.186,110.3,.520,11.9,76.7
